<a href="https://colab.research.google.com/github/junaid537/pytorch/blob/main/%E2%80%9CHW2_Junaid_Khalidi_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


<ipython-input-2-dd748172f966>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


In [3]:
#Ensure that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df = df.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df['star_rating'] = df['star_rating'].astype(int)

<ipython-input-3-251f533e7f5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_rating'] = df['star_rating'].astype(int)


In [4]:
df

,star_rating,review_body
0,5,Great product.
1,5,"What's to say about this commodity item except, what have we come to in this world.<br />Having the need to bnuy captured and compressed air. &#60;lol&#62;"
2,5,"Haven't used yet, but I am sure I will like it."
3,1,"Although this was labeled as &#34;new&#34; the one I received clearly had been used. The box had previously been opened., and the shredder was dirty and the bin was partially full of shredded paper. What was worse is that the unit will not work properly. It is not possible to insert the paper bin so as to enable the shredder to run. It will not operate if the bin is not in place, but I could never get the unit to recognize that the paper bin was actually fully inserted. After cleaning everything thoroughly and vacuuming the paper bin area, it worked ONCE! After that I was unable to get it work at all. I returned the unit immediately for a refund. I feel Amazon misrepresented the unit as &#34;new&#34; when clearly it was not."
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,"I can't live anymore whithout my Palm III. But I also hate to carry things around. Enter the Belt Clip Case for PalmPilot. It is extremely light, holds thightly to my belt or jeans pocket and it features a hard plastic screen protection sewed inside. I let my Palm III (inside this case, fortunately) slip and it fell all the way down on a 17 steps stairs. Not a single scratch! I wish it only had an internal pocket for credit and business cards."
2640348,4,"Although the Palm Pilot is thin and compact it may exceed your pocket capacity. With the belt clip you can carry the Palm Pilot, enter and review data at any time. This especially useful for physicians who are on the move constantly. I have found this carrying case to be an excellent accessory."
2640349,4,This book had a lot of great content without being boring or monotonous. I really enjoy Greek mythology and this book summed it up throughly.
2640350,5,"I am teaching a course in Excel and am using the step-by-step books. They are very good. The class is composed of people who are proficient in 1-2-3, and are computer literate, so we're off and running!"


In [5]:
balanced_df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
balanced_df

<ipython-input-5-7afa5d846570>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


,star_rating,review_body
0,1,"This item was absolutely terrible. The ink barely showed out of my brother printer. I wanted to return it, but it was too late to return. Never buying again..."
1,1,out from the box there is a blank area when it print.
2,1,"I read ALL the reviews Bad & Good and made my Choice, Wrong Choice!!! I really, Really Love Amazon.com But they really need to Use some of these Catridges for Theirself and Evaluate Them. my Printer was Printing Fine it just needed Ink so I looked these up on Amazon and took a Chance even with more than a few Negative Reviews. I sent some Samples of the Horrible attempt at printing in the Box With The easy return for Refund. Went to WalMart and picked up 2 combo packs of Kodak ink for my Two Printers and Sure Enough it went back to Printing Perfect, Both of the Printers. Well I guess the Fact that Kodak's Ink is so Cheap that It has a tendantcy to push you to see just how cheap you can get it for. Please Folks Save your Time and Frustrations for more Important Troubles in your life, Don't Be Greedy. Buy the Kodak Ink you Won't be Dissapointed. Thanks Amazon, for Your easy Returns, because with out them I would really be up the Creek without a Paddle. I just can't say enough, \\""How much I love Amazon.com\\"" Thanks,"
3,1,This came very sharp and it would not come apart. I would not recommend buying this because it cut my finger and it came made wrong. This broke in one day and I regret buying it. Waste of money. :(
4,1,"I'd give this day timer zero stars if that was an option. This calendar is 2 years old! It does say 2011 in the fine print - but being the sucker buyer I did not realize I'd have to verify if it was current - and I paid almost $20 for this old, useless calendar."
...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUEN PRODUCTO LO RECOMIENDO PARA PERSONAS QUE POSEEN SUS OFICINAS A
249996,5,"I have had Panasonic 3 phones for a long time. Recently I had no phones, internet, or security system. I decided to start with new phones. After reading all of the reviews I decided this was the phone system I needed. They came very fast. Still no phone. The phone company came out and found a broken cable leading into my house. Once this was fixed I instantly had phones, internet, & security system. I wish I had ordered a four phone system as I found another area I would like a phone. So, checked with Amazon and instantly found one. It arrived before scheduled delivery date. I love the \\""talking\\"" feature. I do not bother to answer my phone when it tells me \\""out of area\\"". I am sure my older phones still work!* Love Panasonic & Amazon!."
249997,5,These are great pens to draw with and I like the softness of the brush side and the fine tip on the other.
249998,5,"I do martial arts, Jiu-Jitsu and I like to take notes and this book is sturdy and it can take a beating in the room when I am throwing it around after the instructional to the side before drilling. It is great for taking notes in general and good solid book. You can definitely defend yourself with this thing. If this book can handle my roughness it can handle your basic needs, will buy again."


In [6]:
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

balanced_df['sentiment'] = balanced_df['star_rating'].apply(map_sentiment)
balanced_df

,star_rating,review_body,sentiment
0,1,"This item was absolutely terrible. The ink barely showed out of my brother printer. I wanted to return it, but it was too late to return. Never buying again...",2
1,1,out from the box there is a blank area when it print.,2
2,1,"I read ALL the reviews Bad & Good and made my Choice, Wrong Choice!!! I really, Really Love Amazon.com But they really need to Use some of these Catridges for Theirself and Evaluate Them. my Printer was Printing Fine it just needed Ink so I looked these up on Amazon and took a Chance even with more than a few Negative Reviews. I sent some Samples of the Horrible attempt at printing in the Box With The easy return for Refund. Went to WalMart and picked up 2 combo packs of Kodak ink for my Two Printers and Sure Enough it went back to Printing Perfect, Both of the Printers. Well I guess the Fact that Kodak's Ink is so Cheap that It has a tendantcy to push you to see just how cheap you can get it for. Please Folks Save your Time and Frustrations for more Important Troubles in your life, Don't Be Greedy. Buy the Kodak Ink you Won't be Dissapointed. Thanks Amazon, for Your easy Returns, because with out them I would really be up the Creek without a Paddle. I just can't say enough, \\""How much I love Amazon.com\\"" Thanks,",2
3,1,This came very sharp and it would not come apart. I would not recommend buying this because it cut my finger and it came made wrong. This broke in one day and I regret buying it. Waste of money. :(,2
4,1,"I'd give this day timer zero stars if that was an option. This calendar is 2 years old! It does say 2011 in the fine print - but being the sucker buyer I did not realize I'd have to verify if it was current - and I paid almost $20 for this old, useless calendar.",2
...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUEN PRODUCTO LO RECOMIENDO PARA PERSONAS QUE POSEEN SUS OFICINAS A,1
249996,5,"I have had Panasonic 3 phones for a long time. Recently I had no phones, internet, or security system. I decided to start with new phones. After reading all of the reviews I decided this was the phone system I needed. They came very fast. Still no phone. The phone company came out and found a broken cable leading into my house. Once this was fixed I instantly had phones, internet, & security system. I wish I had ordered a four phone system as I found another area I would like a phone. So, checked with Amazon and instantly found one. It arrived before scheduled delivery date. I love the \\""talking\\"" feature. I do not bother to answer my phone when it tells me \\""out of area\\"". I am sure my older phones still work!* Love Panasonic & Amazon!.",1
249997,5,These are great pens to draw with and I like the softness of the brush side and the fine tip on the other.,1
249998,5,"I do martial arts, Jiu-Jitsu and I like to take notes and this book is sturdy and it can take a beating in the room when I am throwing it around after the instructional to the side before drilling. It is great for taking notes in general and good solid book. You can definitely defend yourself with this thing. If this book can handle my roughness it can handle your basic needs, will buy again.",1


In [7]:
#train_df, test_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['sentiment'], random_state=42)


In [8]:
pip install gensim

In [9]:
import pandas as pd
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import nltk

# Download NLTK tokenizer
nltk.download('punkt')
import gensim.downloader as api
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

# Download tokenizer
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
# Load pretrained Word2Vec model (Google News 300)
wv = api.load("word2vec-google-news-300")

# Check the vector representation of a word
#print(wv["excellent"])  # A 300-dimensional vector

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [11]:
#check semantic similarities
result = wv.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("King - Man + Woman =", result[0][0])  # Expected output: 'queen'

King - Man + Woman = queen


In [12]:
#similarities between words
similarity = wv.similarity("excellent", "outstanding")
print("Similarity between 'excellent' and 'outstanding':", similarity)

Similarity between 'excellent' and 'outstanding': 0.55674857


In [14]:
import nltk
nltk.download('punkt_tab')
#tokenize your dataset
# Tokenize the review text
df["tokens"] = df["review_body"].astype(str).apply(word_tokenize)
# Show some tokenized reviews
print(df["tokens"].head())


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      [Great, product, .]
1                                                                                                                                                                                                                                                                                                                                                                     [What, 's, to, say, about, this, commodity, item, except, ,, what, have

In [16]:
#converting to word embeddings
def get_word2vec_embedding(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # 300D vector

# Applying to dataset using the pretrained Word2Vec model
df["embedding"] = df["tokens"].apply(lambda x: get_word2vec_embedding(x, wv))




###(b) Training Word2Vec model on our own datset